### Импорты, параметры ColBERT

In [1]:
from interface import prepare_tsv, save_index, top_n_similar, Collection
import pandas as pd
import os

ckpt_pth = "/home/sondors/Documents/ColBERT_weights/triples_X1_13_categories_use_ib_negatives/none/2024-01/26/10.49.44/checkpoints/colbert-5387-finish"
experiment = "colbert-5387"

doc_maxlen = 300
nbits = 2   # bits определяет количество битов у каждого измерения в семантическом пространстве во время индексации
nranks = 1  # nranks определяет количество GPU для использования, если они доступны
kmeans_niters = 4 # kmeans_niters указывает количество итераций k-means кластеризации; 4 — хороший и быстрый вариант по умолчанию.  

### Подготовка моделей

В данном примере готовим модели в качестве того на что матчить оффера. Данные должны быть в формате, пригодном для индексации с помощью ColBERT. В общем случае в индексе может быть и смесь моделей и уже ранее моделизованных офферов

In [2]:
pth_models = "/home/sondors/Documents/price/ColBERT_data/18_categories/test/models_18_categories.csv"
df_models = pd.read_csv(pth_models, sep=";")
df_models

/tmp/ipykernel_60661/17807022.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_models = pd.read_csv(pth_models, sep=";")


,model_id,average_price,name,brand_name,full_name,comment,category_name,category_id
0,623742,NaN,920-005619,Logitech,Logitech 920-005619,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
1,721952,NaN,Zipper Bag,Hama,Hama Zipper Bag,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
2,721970,NaN,CC-3064,Nokia,Nokia CC-3064,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
3,751488,990.0,CKS-X7/R,Sony,Sony CKS-X7/R,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
4,751989,NaN,EP-031023,Era Pro,Era Pro EP-031023,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
...,...,...,...,...,...,...,...,...
103209,7049424,16459.0,MD-108,Mivo,Mivo MD-108,NaN,портативная акустика,3904
103210,7049425,8812.0,MD-165,Mivo,Mivo MD-165,NaN,портативная акустика,3904
103211,7049426,4240.0,Boost 20W,Rocket,Rocket Boost 20W,NaN,портативная акустика,3904
103212,7049427,2990.0,Motion 10W,Rocket,Rocket Motion 10W,NaN,портативная акустика,3904


In [3]:
id_category = {
    3902: 'диктофоны, портативные рекордеры',
    510402: 'электронные книги',
    4302: 'автомобильные телевизоры, мониторы',
    2815: 'смарт-часы и браслеты',
    3901: 'портативные медиаплееры',
    3904: 'портативная акустика',
    2801: 'мобильные телефоны',
    3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
    510401: 'планшетные компьютеры и мини-планшеты',
    2102: 'наушники, гарнитуры, наушники c микрофоном',
    3903: 'радиоприемники, радиобудильники, радиочасы',
    3907: 'магнитолы',
    280801: 'GPS-навигаторы'
    }

dst_fld = "/home/sondors/Documents/price/ColBERT/tutorial"
for cat_id in id_category.keys():
    category_models = df_models[df_models.category_id == cat_id].reset_index(drop=True)
    prepare_tsv(category_models, dst_fld, cat_id)

### Индексируем модели и сохраняем индекс на диск

В id_category всего 13 категорий, в сумме 41037 шт моделей:

- ColBERT bert-base-multilingual-cased_dim_768: индексация за 760 сек на CPU с 16 ядрами без GPU, вес индекса 109 Мб

- ColBERTv2.0_dim_128: индексация за 790 сек на CPU с 16 ядрами без GPU, вес индекса 20.6 Мб

In [4]:
for cat_id in id_category.keys():
    models_colbert = Collection(path=os.path.join(dst_fld, "tsv", f"{cat_id}_models.tsv"))
    index_name = f'models_{cat_id}_{nbits}bits'
    indexer = save_index(ckpt_pth, doc_maxlen, nbits, kmeans_niters, nranks, dst_fld, experiment, models_colbert, index_name)

[Feb 07, 20:48:20] #> Loading collection...
0M 


[Feb 07, 20:48:20] #> Creating directory /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_3902_2bits 


#> Starting...
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "nbits": 2,
    "kmeans_niters": 4,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 0.0001,
    "maxsteps": 5387,
    "save_every": null,
    "warmup": 0,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 300,
    "mask_punctua

/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
  0%|          | 0/8 [00:00<?, ?it/s]/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Feb 07, 20:48:24] [0] 		 # of sampled PIDs = 488 	 sampled_pids[:3] = [213, 375, 5]
[Feb 07, 20:48:24] [0] 		 #> Encoding 488 passages..


100%|██████████| 8/8 [00:06<00:00,  1.17it/s]
WARNING clustering 4558 points to 1024 centroids: please provide at least 39936 training points
0it [00:00, ?it/s]
  0%|          | 0/8 [00:00<?, ?it/s]

[Feb 07, 20:48:31] [0] 		 avg_doclen_est = 9.829917907714844 	 len(local_sample) = 488
[Feb 07, 20:48:31] [0] 		 Creaing 1,024 partitions.
[Feb 07, 20:48:31] [0] 		 *Estimated* 4,796 embeddings.
[Feb 07, 20:48:31] [0] 		 #> Saving the indexing plan to /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_3902_2bits/plan.json ..
Clustering 4558 points in 128D to 1024 clusters, redo 1 times, 4 iterations
  Preprocessing in 0.00 s
  Iteration 3 (0.01 s, search 0.01 s): objective=409.356 imbalance=1.531 nsplit=0       
[0.022, 0.023, 0.02, 0.021, 0.021, 0.021, 0.023, 0.021, 0.017, 0.019, 0.019, 0.023, 0.02, 0.021, 0.02, 0.026, 0.02, 0.021, 0.022, 0.02, 0.021, 0.026, 0.022, 0.018, 0.02, 0.021, 0.023, 0.023, 0.024, 0.019, 0.02, 0.019, 0.018, 0.018, 0.024, 0.022, 0.019, 0.022, 0.022, 0.022, 0.02, 0.022, 0.023, 0.017, 0.021, 0.019, 0.018, 0.019, 0.019, 0.021, 0.025, 0.021, 0.024, 0.019, 0.022, 0.017, 0.018, 0.021, 0.023, 0.02, 0.019, 0.021, 0.024, 0.023, 0.019, 0.022, 0.01

...

[Feb 07, 21:01:00] [0] 		 # of sampled PIDs = 1709 	 sampled_pids[:3] = [853, 1500, 20]
[Feb 07, 21:01:00] [0] 		 #> Encoding 1709 passages..


100%|██████████| 27/27 [00:15<00:00,  1.77it/s]
WARNING clustering 14954 points to 1024 centroids: please provide at least 39936 training points
0it [00:00, ?it/s]
  0%|          | 0/27 [00:00<?, ?it/s]

[Feb 07, 21:01:15] [0] 		 avg_doclen_est = 9.210649490356445 	 len(local_sample) = 1,709
[Feb 07, 21:01:15] [0] 		 Creaing 1,024 partitions.
[Feb 07, 21:01:15] [0] 		 *Estimated* 15,740 embeddings.
[Feb 07, 21:01:15] [0] 		 #> Saving the indexing plan to /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_280801_2bits/plan.json ..
Clustering 14954 points in 128D to 1024 clusters, redo 1 times, 4 iterations
  Preprocessing in 0.00 s
  Iteration 3 (0.04 s, search 0.04 s): objective=3085.4 imbalance=1.547 nsplit=0        
[0.032, 0.031, 0.029, 0.032, 0.034, 0.032, 0.031, 0.034, 0.028, 0.032, 0.031, 0.036, 0.032, 0.035, 0.031, 0.036, 0.03, 0.031, 0.028, 0.03, 0.031, 0.034, 0.032, 0.03, 0.03, 0.035, 0.034, 0.037, 0.034, 0.032, 0.03, 0.031, 0.032, 0.031, 0.034, 0.036, 0.029, 0.03, 0.037, 0.029, 0.03, 0.033, 0.036, 0.032, 0.034, 0.032, 0.032, 0.029, 0.032, 0.03, 0.032, 0.031, 0.034, 0.03, 0.035, 0.029, 0.034, 0.029, 0.034, 0.034, 0.03, 0.03, 0.035, 0.038, 0.032, 0.03, 0


100%|██████████| 27/27 [00:15<00:00,  1.74it/s]
1it [00:15, 15.61s/it]
100%|██████████| 1024/1024 [00:00<00:00, 137959.89it/s]


[Feb 07, 21:01:31] [0] 		 #> Saving chunk 0: 	 1,709 passages and 15,741 embeddings. From #0 onward.
[Feb 07, 21:01:31] [0] 		 #> Checking all files were saved...
[Feb 07, 21:01:31] [0] 		 Found all files!
[Feb 07, 21:01:31] [0] 		 #> Building IVF...
[Feb 07, 21:01:31] [0] 		 #> Loading codes...
[Feb 07, 21:01:31] [0] 		 Sorting codes...
[Feb 07, 21:01:31] [0] 		 Getting unique codes...
[Feb 07, 21:01:31] #> Optimizing IVF to store map from centroids to list of pids..
[Feb 07, 21:01:31] #> Building the emb2pid mapping..
[Feb 07, 21:01:31] len(emb2pid) = 15741
[Feb 07, 21:01:31] #> Saved optimized IVF to /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_280801_2bits/ivf.pid.pt
[Feb 07, 21:01:31] [0] 		 #> Saving the indexing metadata to /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_280801_2bits/metadata.json ..
#> Joined...


### Поиск матча по индексу

Запуск 77к офферов на поиск матча среди 13 категорий или 41к шт моделей занимает 20 мин на CPU

In [5]:
offers = {
    0: 'Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)',
    1: 'Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)',
    2: 'Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)'
    }

src_fld = "/home/sondors/Documents/price/ColBERT/tutorial"
cat_id = 510401 # планшеты
index_name = f'models_{cat_id}_{nbits}bits'
models_id_colbert = Collection(path=os.path.join(src_fld, "tsv", f"{cat_id}_models_id.tsv"))

top_n = top_n_similar(offers, src_fld, nranks, experiment, index_name, models_id_colbert, 5)

[Feb 07, 21:01:32] #> Loading collection...
0M 
[Feb 07, 21:01:32] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Feb 07, 21:01:32] #> Loading codec...
[Feb 07, 21:01:32] #> Loading IVF...
[Feb 07, 21:01:32] Loading segmented_lookup_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


[Feb 07, 21:01:33] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 3032.76it/s]

[Feb 07, 21:01:33] #> Loading codes and residuals...


...

[Feb 07, 21:01:33] Loading filter_pids_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


[Feb 07, 21:01:33] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')



#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global), 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1, 19102,  1194, 29436, 28995, 29753, 15290, 22919, 19102,
         9088, 21628,  1055,  2620,  1010,  1022,  1183, 29740,  1013, 11899,
         1183, 29740,  1010, 15536,  1011, 10882,  1009, 12562,  1010,  1196,
        14150,   102])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])



3it [00:00, 367.22it/s]


### Получаем выдачу топ N моделей для каждого оффера

In [6]:
for i in range(len(top_n)):
    print(offers[i])
    for j in range(len(top_n[i]['model_ids'])):
        id = top_n[i]['model_ids'][j]
        sim = top_n[i]['similarity'][j]
        model = list(df_models[df_models.model_id == id]['full_name'])[0]
        print(f"\t{id}: {model} --> {round(float(sim), 2)}")
    print("_"*60)

Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)
	5144478: Samsung Galaxy Tab S8 --> 25.51
	5144479: Samsung Galaxy Tab S8 Ultra --> 22.29
	3843929: Samsung Galaxy Tab S4 10.5 SM-T835 --> 21.06
	4892891: Samsung Galaxy Tab A8 10.5 128Gb --> 19.64
	623215: Samsung Galaxy Tab S 8.4 SM-T700 --> 19.64
____________________________________________________________
Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)
	5144478: Samsung Galaxy Tab S8 --> 25.12
	5144479: Samsung Galaxy Tab S8 Ultra --> 21.7
	3843929: Samsung Galaxy Tab S4 10.5 SM-T835 --> 21.3
	1464907: Samsung Galaxy Tab S3 9.7 SM-T820 --> 19.38
	623215: Samsung Galaxy Tab S 8.4 SM-T700 --> 19.34
____________________________________________________________
Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)
	5144477: Samsung Galaxy Tab S8+ --> 23.26
	4509798: Samsung Galaxy Tab S7+ 12.4 128Gb --> 19.52
	5144478: Samsung Galaxy Tab S8 --> 19.51
	623213: Samsung Gala

In [ ]:
"""
Для эксперимента bert-base-multilingual-cased-2998:

Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)
	5144478: Samsung Galaxy Tab S8 --> 28.97
	5144477: Samsung Galaxy Tab S8+ --> 24.97
	937550: Samsung Galaxy Tab S2 9.7 SM-T813 --> 24.0
	5144479: Samsung Galaxy Tab S8 Ultra --> 23.98
	816746: Samsung Galaxy Tab S2 9.7 SM-T810 --> 23.64
____________________________________________________________
Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)
	5144478: Samsung Galaxy Tab S8 --> 28.15
	5144477: Samsung Galaxy Tab S8+ --> 23.94
	937550: Samsung Galaxy Tab S2 9.7 SM-T813 --> 23.21
	623216: Samsung Galaxy Tab S 8.4 SM-T705 --> 22.82
	816746: Samsung Galaxy Tab S2 9.7 SM-T810 --> 22.65
____________________________________________________________
Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)
	5144477: Samsung Galaxy Tab S8+ --> 26.03
	5144478: Samsung Galaxy Tab S8 --> 20.32
	4509798: Samsung Galaxy Tab S7+ 12.4 128Gb --> 18.63
	937550: Samsung Galaxy Tab S2 9.7 SM-T813 --> 18.35
	5144479: Samsung Galaxy Tab S8 Ultra --> 18.31
____________________________________________________________
"""

In [7]:
top_n

[{'model_ids': [5144478, 5144479, 3843929, 4892891, 623215],
  'similarity': [25.507810592651367,
   22.290096282958984,
   21.059688568115234,
   19.644437789916992,
   19.64250946044922]},
 {'model_ids': [5144478, 5144479, 3843929, 1464907, 623215],
  'similarity': [25.12063980102539,
   21.704925537109375,
   21.299270629882812,
   19.382617950439453,
   19.33898162841797]},
 {'model_ids': [5144477, 4509798, 5144478, 623213, 5144479],
  'similarity': [23.25652503967285,
   19.523971557617188,
   19.506370544433594,
   19.483131408691406,
   18.18914794921875]}]